In [1]:
import pandas as pd
import numpy as np

from tensorflow import keras
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor, KerasClassifier
from tensorflow.keras import losses
from tensorflow.keras.constraints import MaxNorm as maxnorm
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_validate
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import pickle
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
#from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, roc_auc_score
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import Audio
#sound_file = 'beep.wav'
from tensorflow.keras.callbacks import *


import numpy as np
import pandas as pd   
import matplotlib.pyplot as plt
import random
import os

from sklearn.metrics import mean_absolute_error

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import * 
from tensorflow.keras.optimizers import *
from tensorflow.keras import backend as K

In [2]:
class T2V(Layer):
    
    def __init__(self, output_dim=None, **kwargs):
        self.output_dim = output_dim
        super(T2V, self).__init__(**kwargs)
        
    def build(self, input_shape):

        self.W = self.add_weight(name='W',
                                shape=(input_shape[-1], self.output_dim),
                                initializer='uniform',
                                trainable=True)

        self.P = self.add_weight(name='P',
                                shape=(input_shape[1], self.output_dim),
                                initializer='uniform',
                                trainable=True)

        self.w = self.add_weight(name='w',
                                shape=(input_shape[1], 1),
                                initializer='uniform',
                                trainable=True)

        self.p = self.add_weight(name='p',
                                shape=(input_shape[1], 1),
                                initializer='uniform',
                                trainable=True)

        super(T2V, self).build(input_shape)
        
    def call(self, x):
        
        original = self.w * x + self.p
        sin_trans = K.sin(K.dot(x, self.W) + self.P)
        
        return K.concatenate([sin_trans, original], -1)


In [3]:
np.random.seed(7)

In [174]:
# load dataset

#dataframe = pd.read_csv("../data/bitcoin_ticker.csv", sep=',')
dataframe = pd.read_csv("../data/new/features_regression_boruta_plus_rf.csv", sep=',')

In [132]:
#reg_seven = ["difficulty30rsi", "difficulty90var","fee_to_reward90rsiUSD","hashrate30var","median_transaction_fee7rocUSD","mining_profitability","price30smaUSD","price3wmaUSD","price7wmaUSD","sentinusd90emaUSD","size90trx","top100cap","transactionvalueUSD","priceUSD"]
#dataframe = dataframe[reg_seven]
dataframe.head(3)
#dataframe = dataframe.drop(['Date'], axis = 1)

In [133]:
# # y_train_pred=regressor.predict(X_train)
# # #X_train[0:]
# # #list(zip(*X_train))[0]
# # y_train.shape
# #dataframe.describe()

length=dataframe.shape[1]-1
df = dataframe

for i in dataframe.columns:
    try:
        dataframe[i] = np.where(dataframe[i] > df[i].quantile(0.75), df[i].quantile(0.50), df[i])
        dataframe[i] = np.where(dataframe[i] < df[i].quantile(0.25), df[i].quantile(0.50), df[i])
    except:
        pass


# from scipy import stats
# import numpy as np
# z = np.abs(stats.zscore(dataframe))

# Q1 = dataframe.quantile(0.25)
# Q3 = dataframe.quantile(0.75)
# IQR = Q3 - Q1


# df_o = dataframe[(z < 3).all(axis=1)]
# df_out = df_o[~((df_o < (Q1 - 1.5 * IQR)) |(df_o > (Q3 + 1.5 * IQR))).any(axis=1)]
# dataframe = df_out

In [134]:
set(dataframe['priceUSD'])

{86.59100000000002,
 87.454,
 87.10799999999998,
 89.05,
 89.28299999999999,
 91.849,
 92.446,
 86.48299999999998,
 94.715,
 94.679,
 87.875,
 97.247,
 98.435,
 98.478,
 100.25,
 100.355,
 102.694,
 103.663,
 103.671,
 104.575,
 105.04,
 105.104,
 105.826,
 105.11,
 105.799,
 106.7,
 106.072,
 106.115,
 107.749,
 107.292,
 107.66,
 108.42,
 109.595,
 110.375,
 120.405,
 112.503,
 119.368,
 123.693,
 115.0,
 116.686,
 126.316,
 118.696,
 120.096,
 120.003,
 124.135,
 124.605,
 125.725,
 126.909,
 126.424,
 128.664,
 128.75,
 130.56,
 132.945,
 134.114,
 134.8,
 134.465,
 139.74,
 134.739,
 136.214,
 142.08,
 138.704,
 143.72299999999996,
 153.886,
 157.329,
 163.561,
 164.81,
 169.46900000000005,
 172.97,
 178.5,
 178.428,
 180.316,
 185.605,
 188.042,
 191.426,
 193.134,
 195.397,
 195.57,
 195.73,
 197.87,
 199.6,
 197.577,
 200.604,
 202.56,
 205.937,
 205.929,
 210.64,
 211.756,
 211.824,
 213.806,
 214.636,
 213.673,
 218.482,
 219.936,
 220.946,
 218.176,
 222.448,
 220.585,
 224.

In [135]:
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])

    return np.array(dataX), np.array(dataY)

In [175]:
dataset = dataframe[pd.to_datetime(dataframe['Date']) > pd.to_datetime('2017-06-28 00:00:00')]

dataset = dataset[['priceUSD']].values.astype('float32')
# X, y = create_dataset(dataframe[['priceUSD']])
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

# train_size = int(len(dataset) * 0.67)
# test_size = len(dataset) - train_size
# train, test = dataset[0:train_size, :], dataset[train_size:len(dataset), :]
# print(len(train), len(test))

# look_back = 10
# trainX, trainY = create_dataset(train, look_back=look_back)
# testX, testY = create_dataset(test, look_back=look_back)

In [148]:
# # split into input (X) and output (Y) variables
# #X = dataframe.iloc[:,0:length]
# #X = X[["hashrate","size90var","difficulty14sma","difficulty30sma","difficulty90sma","difficulty3ema","difficulty7ema"]]
# X = dataframe#[['price7wmaUSD', 'difficulty90var']]
# y = dataframe['priceUSD']
# #X = X.drop(['Date', 'priceUSD'], axis = 1)
# X = X.drop(['priceUSD'], axis = 1)

In [149]:
# X.head(3)
#X = X.drop(['Date', 'priceUSD'], axis = 1)
# #X = dataframe.iloc[:,:-1]

In [150]:
#y=dataframe.iloc[:,-1:]

In [151]:
# shape=X.shape[1]

In [152]:
dataframe[pd.to_datetime(dataframe['Date']) > pd.to_datetime('2019-06-28 00:00:00')]

,Date,priceUSD,price3smaUSD,price7smaUSD,price14smaUSD,price30smaUSD,price3emaUSD,price7emaUSD,price14emaUSD,price30emaUSD,price90emaUSD,price3wmaUSD,price7wmaUSD,price14wmaUSD,price30wmaUSD,price90wmaUSD,transactionvalue90smaUSD,price3momUSD,price90stdUSD
3269,2019/06/29,11867.0,11798.0,11576.0,10537.0,9253.0,11816.0,11458.0,10725.0,9666.0,7678.0,11805.0,11770.0,11133.0,9960.0,8262.0,29913.0,-800.919,3971.0
3270,2019/06/30,11569.0,11713.0,11688.0,10714.0,9360.0,11693.0,11486.0,10837.0,9789.0,7763.0,11691.0,11768.0,11271.0,10109.0,8358.0,30438.0,-256.912,4022.0
3271,2019/07/01,10709.0,11382.0,11666.0,10820.0,9433.0,11201.0,11291.0,10820.0,9848.0,7828.0,11189.0,11523.0,11270.0,10196.0,8433.0,30739.0,-993.205,4046.0
3272,2019/07/02,10323.0,10867.0,11523.0,10903.0,9488.0,10762.0,11049.0,10754.0,9879.0,7883.0,10659.0,11188.0,11204.0,10254.0,8498.0,30988.0,-1545.000,4063.0
3273,2019/07/03,11337.0,10790.0,11333.0,11060.0,9581.0,11049.0,11121.0,10832.0,9973.0,7959.0,10894.0,11141.0,11261.0,10373.0,8584.0,31328.0,-231.729,4111.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4016,2021/07/15,32109.0,32485.0,33117.0,33598.0,34185.0,32495.0,32977.0,33427.0,34509.0,39609.0,32360.0,32884.0,33295.0,33658.0,37467.0,318035.0,-1592.000,18643.0
4017,2021/07/16,31750.0,32116.0,32915.0,33489.0,33929.0,32123.0,32670.0,33204.0,34331.0,39436.0,31993.0,32542.0,33048.0,33501.0,37239.0,316927.0,-1105.000,18320.0
4018,2021/07/17,31556.0,31805.0,32608.0,33293.0,33694.0,31839.0,32391.0,32984.0,34152.0,39263.0,31713.0,32202.0,32790.0,33348.0,37013.0,316752.0,-933.146,18193.0
4019,2021/07/18,31763.0,31690.0,32318.0,33042.0,33519.0,31801.0,32234.0,32821.0,33998.0,39098.0,31692.0,31991.0,32587.0,33224.0,36798.0,315474.0,-346.009,18031.0


In [215]:
# X_train,X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, train_size=0.8, shuffle=True, random_state=7)

In [216]:
#estimators=[]

In [217]:
#estimators.append(['mixmax',MinMaxScaler()])

In [218]:
#estimators.append(['robust',RobustScaler()])

In [219]:
#scale=Pipeline(estimators,verbose=True)

In [220]:

# X_train,X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, train_size=0.8, shuffle=True, random_state=7)

# X_train=scale.fit_transform(X_train)
# X_test=scale.transform(X_test)

# from sklearn.neighbors import LocalOutlierFactor

# lof = LocalOutlierFactor()
# yhat = lof.fit_predict(X_train)
# mask = yhat != -1
# X_train, y_train = X_train[mask, :], y_train[mask]

# X_train=np.reshape(X_train,(X_train.shape[0],1,X_train.shape[1]))
# X_test=np.reshape(X_test,(X_test.shape[0],1,X_test.shape[1]))

# y_train=y_train.values

# y_train=np.reshape(y_train, (y_train.shape[0],1,1))

# y_test=y_test.values

# y_test=np.reshape(y_test,(y_test.shape[0],1,1))

# X_train.shape, y_train.shape

In [221]:
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [25]:
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [61]:
# define neural network model
def sequential_model(initializer='normal', activation='relu', neurons=100, NUM_FEATURES=90):
    # create model
    model = Sequential()
    #model = Sequential()
    #model.add(tf.keras.layers.Reshape((NUM_FEATURES,1), input_shape=(NUM_FEATURES,)))
    model.add(LSTM(neurons, return_sequences=True, activation='relu',input_shape=(1, look_back)))
    model.add(T2V(neurons))
    
    #model.add(LSTM(neurons, activation=activation, input_shape=(NUM_FEATURES,1)))
    #model.add(Bidirectional(LSTM(neurons, return_sequences=True, activation='relu')))
    #model.add(Bidirectional(LSTM(350, return_sequences=True, activation='relu')))
    
    model.add(Dense(neurons, activation=activation))
    model.add(Dense(1, activation=activation))
    #model.compile(loss="logcosh", optimizer=adam, metrics=['mae'])
#    model.add(Dense(neurons, input_shape=(NUM_FEATURES,), kernel_initializer=initializer, activation=activation))
#     model.add(Dense(neurons, activation=activation))
#     model.add(Dense(neurons, activation=activation))
    #model.add(tf.keras.layers.Reshape((NUM_FEATURES,1), input_shape=(NUM_FEATURES,)))
    #model.add(T2V(neurons))
    #model.add(LSTM(neurons, activation=activation, input_shape=(NUM_FEATURES,1)))
    #model.add(T2V(neurons))
#     model.add(Dense(1, activation=activation, kernel_initializer=initializer))
#     model.add(T2V(neurons))
#     model.add(Dense(1, activation=activation))
    model.add(Dropout(0.5))
    #model.add(LSTM(neurons, activation=activation))
    # Compile model
    adam=keras.optimizers.Adam(lr=lr_schedule(100), amsgrad=True)
    #sgd=keras.optimizers.SGD(learning_rate=0.08, momentum=0.9, nesterov=False)
    model.compile(loss='mae', optimizer=adam, metrics=['mae'])
    return model

In [62]:
mcp_save = ModelCheckpoint('ANN_reg_seven_new.hdf5', save_best_only=True, monitor='val_loss', mode='auto')
earlyStopping = EarlyStopping(monitor='val_loss', patience=10,verbose=1, mode='auto')

In [63]:
regressor=KerasRegressor(build_fn=sequential_model, batch_size=64, epochs=100,verbose=1, shuffle=True,
                         validation_split=0.1,validation_freq=1,
                        use_multiprocessing=True) #callbacks=[mcp_save,earlyStopping])

In [64]:

regressor.fit(trainX,trainY)


Learning rate:  0.0001
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 28737 samples, validate on 3194 samples
Epoch 1/100
28737/28737 [==============================] - 5s 160us/sample - loss: 0.4060 - mean_absolute_error: 0.4060 - val_loss: 0.2617 - val_mean_absolute_error: 0.2618
Epoch 2/100
28737/28737 [==============================] - 3s 112us/sample - loss: 0.3928 - mean_absolute_error: 0.3928 - val_loss: 0.2518 - val_mean_absolute_error: 0.2518
Epoch 3/100
28737/28737 [==============================] - 3s 115us/sample - loss: 0.3918 - mean_absolute_error: 0.3918 - val_loss: 0.2691 - val_mean_absolute_error: 0.2691
Epoch 4/100
28737/28737 [==============================] - 3s 108us/sample - loss: 0.3919 - mean_absolute_error: 0.3919 - val_loss: 0.2576 - val_mean_absolute_error: 0.2576
Epoch 5/100
28737/28737 [==============================] - 3s 87us/sample - loss: 0.3924 - mean_absolute_error: 0.3924 - val_loss: 0.2714 - va

KeyboardInterrupt: 

In [60]:
y_pred=regressor.predict(testX)
r2_score(testY, y_pred) #training score

15723/15723 [==============================] - 0s 26us/sample


-5.092442444932622

In [182]:
y_pred=regressor.predict(testX)

1325/1325 [==============================] - 0s 47us/sample


In [198]:
testY

array([0.07285933, 0.07285933, 0.07285933, ..., 0.07285933, 0.07285933,
       0.07285933], dtype=float32)

In [229]:
trainPredict = regressor.predict(trainX)
testPredict = regressor.predict(testX)

1316/1316 [==============================] - 0s 30us/sample


In [231]:
trainPredict = scaler.inverse_transform([trainPredict])
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform([testPredict])
testY = scaler.inverse_transform([testY])

In [253]:
import math
trainScore = math.sqrt(mean_squared_error(trainY, trainPredict))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY, testPredict))
print('Test Score: %.2f RMSE' % (testScore))

Train Score: 977.99 RMSE
Test Score: 2747.66 RMSE


In [263]:
max(testPredict[0])

488.38431680298936

In [9]:
df = dataframe.loc[(dataframe['rpt_key'] == 'btc_usd')]


In [39]:
df = dataframe.loc[(dataframe['rpt_key'] == 'btc_usd')]

df = df.reset_index(drop=True)
df['datetime'] = pd.to_datetime(df['datetime_id'])
df = df.loc[df['datetime'] > pd.to_datetime('2017-06-28 00:00:00')]
df = df[['last']]

dataset = df.values
dataset = dataset.astype('float32')

scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

In [40]:
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size, :], dataset[train_size:len(dataset), :]
print(len(train), len(test))

31942 15734


In [176]:
look_back = 10
trainX, trainY = create_dataset(train, look_back=look_back)
testX, testY = create_dataset(test, look_back=look_back)
# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))
testX = np.reshape(testX, (testX.shape[0], testX.shape[1], 1))

In [114]:
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

In [99]:
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

In [100]:
import math
trainScore = math.sqrt(mean_squared_error(trainY, trainPredict))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY, testPredict))
print('Test Score: %.2f RMSE' % (testScore))

ValueError: Found input variables with inconsistent numbers of samples: [1, 31931]

In [177]:
model = Sequential()
model.add(T2V(4))
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=100, batch_size=256, verbose=2, validation_split=0.1)

Train on 882 samples, validate on 99 samples
Epoch 1/100
882/882 - 5s - loss: 0.0096 - val_loss: 0.0088
Epoch 2/100
882/882 - 0s - loss: 0.0073 - val_loss: 0.0063
Epoch 3/100
882/882 - 0s - loss: 0.0054 - val_loss: 0.0042
Epoch 4/100
882/882 - 0s - loss: 0.0041 - val_loss: 0.0027
Epoch 5/100
882/882 - 0s - loss: 0.0031 - val_loss: 0.0016
Epoch 6/100
882/882 - 0s - loss: 0.0025 - val_loss: 8.8123e-04
Epoch 7/100
882/882 - 0s - loss: 0.0023 - val_loss: 5.3160e-04
Epoch 8/100
882/882 - 0s - loss: 0.0023 - val_loss: 3.9229e-04
Epoch 9/100
882/882 - 0s - loss: 0.0024 - val_loss: 3.5378e-04
Epoch 10/100
882/882 - 0s - loss: 0.0024 - val_loss: 3.5263e-04
Epoch 11/100
882/882 - 0s - loss: 0.0023 - val_loss: 3.7655e-04
Epoch 12/100
882/882 - 0s - loss: 0.0023 - val_loss: 4.2604e-04
Epoch 13/100
882/882 - 0s - loss: 0.0022 - val_loss: 4.9141e-04
Epoch 14/100
882/882 - 0s - loss: 0.0022 - val_loss: 5.5214e-04
Epoch 15/100
882/882 - 0s - loss: 0.0022 - val_loss: 6.0101e-04
Epoch 16/100
882/882 - 0

In [182]:
look_back = 10
# trainX, trainY = create_dataset(train, look_back=look_back)
# testX, testY = create_dataset(test, look_back=look_back)
# # reshape input to be [samples, time steps, features]
# trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))
# testX = np.reshape(testX, (testX.shape[0], testX.shape[1], 1))

trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:, 0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:, 0]))
print('Test Score: %.2f RMSE' % (testScore))

Train Score: 664.73 RMSE
Test Score: 2940.86 RMSE


In [181]:
trainPredict

array([[0.01584443],
       [0.01586335],
       [0.01564194],
       [0.01516136],
       [0.01479109],
       [0.01451432],
       [0.0139946 ],
       [0.01301383],
       [0.01181395],
       [0.01118748],
       [0.01128623],
       [0.01143672],
       [0.01204608],
       [0.01326329],
       [0.01443493],
       [0.01536917],
       [0.01614139],
       [0.01652208],
       [0.01656823],
       [0.01676511],
       [0.01731792],
       [0.01769279],
       [0.01783619],
       [0.01809091],
       [0.01831516],
       [0.01830586],
       [0.01839006],
       [0.01875017],
       [0.02002098],
       [0.02127182],
       [0.02240076],
       [0.02369437],
       [0.02458458],
       [0.02541979],
       [0.02645486],
       [0.02808263],
       [0.03017146],
       [0.0322651 ],
       [0.0338631 ],
       [0.03514253],
       [0.03669856],
       [0.03760789],
       [0.03794946],
       [0.03827435],
       [0.03832726],
       [0.03794815],
       [0.03814811],
       [0.038

In [172]:
trainPredict

array([[ 2978.1992],
       [ 2977.6848],
       [ 2956.6138],
       [ 2913.8787],
       [ 2882.6467],
       [ 2866.0913],
       [ 2832.9712],
       [ 2760.31  ],
       [ 2672.5784],
       [ 2638.3323],
       [ 2669.8494],
       [ 2705.6697],
       [ 2772.752 ],
       [ 2877.315 ],
       [ 2968.362 ],
       [ 3030.7207],
       [ 3069.2244],
       [ 3066.5845],
       [ 3034.7637],
       [ 3027.8677],
       [ 3065.6833],
       [ 3090.7773],
       [ 3099.7214],
       [ 3121.4614],
       [ 3139.6707],
       [ 3136.4849],
       [ 3140.745 ],
       [ 3165.4932],
       [ 3267.1196],
       [ 3362.074 ],
       [ 3446.8699],
       [ 3533.698 ],
       [ 3577.139 ],
       [ 3618.7925],
       [ 3676.285 ],
       [ 3787.9783],
       [ 3936.243 ],
       [ 4079.9216],
       [ 4179.268 ],
       [ 4243.6177],
       [ 4333.1406],
       [ 4366.1733],
       [ 4352.63  ],
       [ 4342.697 ],
       [ 4319.8477],
       [ 4276.7183],
       [ 4299.7295],
       [ 4327